## 라이브러리

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import csv
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'NanumGothic'

## vvt 파일 불러오기

In [6]:
select_category = 'Drama' # 카테고리를 선택해주세요!! 

_list = os.listdir(f'../script/{select_category}')
title_list = []

for title in _list:
    if title.startswith('.'): continue
    title_list.append(title)
title_list = pd.DataFrame(title_list)
title_list = title_list.rename(columns = {0 : 'title'})

title_list

,title
0,Anne.with.an.E
1,Better.Call.Saul
2,Black.Mirror
3,Breaking.Bad
4,Disenchantment
5,Friends
6,How.to.Get.Away.With.Murder
7,MINDHUNTER
8,Narcos
9,Peaky.Blinders


# 위의 리스트에서 작품 인덱스를 넣으세요

In [176]:
title_num = 13 # 작품 인덱스 넣는 곳

In [177]:
_list = os.listdir(f'../script/{select_category}/' + title_list.loc[title_num]['title'])
file_list = []
for file in _list:
    if file.startswith('.'): continue
    if file[-3:] == 'csv': continue
    file_list.append(file)
file_list

['The.Good.Doctor.S01E01.WEBRip.Netflix.en[cc].vtt',
 'The.Good.Doctor.S01E02.WEBRip.Netflix.en[cc].vtt',
 'The.Good.Doctor.S01E03.WEBRip.Netflix.en[cc].vtt',
 'The.Good.Doctor.S01E04.WEBRip.Netflix.en[cc].vtt',
 'The.Good.Doctor.S01E05.WEBRip.Netflix.en[cc].vtt',
 'The.Good.Doctor.S01E06.WEBRip.Netflix.en[cc].vtt',
 'The.Good.Doctor.S01E07.WEBRip.Netflix.en[cc].vtt',
 'The.Good.Doctor.S01E08.WEBRip.Netflix.en[cc].vtt',
 'The.Good.Doctor.S01E09.WEBRip.Netflix.en[cc].vtt',
 'The.Good.Doctor.S01E10.WEBRip.Netflix.en[cc].vtt']

In [178]:
title = title_list.loc[title_num]['title']

path = f'../script/{select_category}/' + title_list.loc[title_num]['title']

full_segments = ''

for file_name in file_list:
    print(file_name)
    opened_file = open(f'{path}/{file_name}', encoding='utf8')
    content = opened_file.read()
    segments = content.split('NOTE /SegmentIndex')[1]    # 한줄씩 정리

    full_segments += segments

segments = full_segments.split('\n\n')

The.Good.Doctor.S01E01.WEBRip.Netflix.en[cc].vtt
The.Good.Doctor.S01E02.WEBRip.Netflix.en[cc].vtt
The.Good.Doctor.S01E03.WEBRip.Netflix.en[cc].vtt
The.Good.Doctor.S01E04.WEBRip.Netflix.en[cc].vtt
The.Good.Doctor.S01E05.WEBRip.Netflix.en[cc].vtt
The.Good.Doctor.S01E06.WEBRip.Netflix.en[cc].vtt
The.Good.Doctor.S01E07.WEBRip.Netflix.en[cc].vtt
The.Good.Doctor.S01E08.WEBRip.Netflix.en[cc].vtt
The.Good.Doctor.S01E09.WEBRip.Netflix.en[cc].vtt
The.Good.Doctor.S01E10.WEBRip.Netflix.en[cc].vtt


In [179]:
display(len(segments), segments[:5])

12362

['',
 '                                                                                                                                                                                         ',
 '\n1\n00:00:03.420 --> 00:00:12.847 position:59.52%,start align:start size:13.81% line:10.00% \n<c.white><c.mono_sans>♪♪</c.mono_sans></c.white>',
 '2\n00:00:12.888 --> 00:00:22.356 position:59.52%,start align:start size:13.81% line:10.00% \n<c.white><c.mono_sans>♪♪</c.mono_sans></c.white>',
 '3\n00:00:22.397 --> 00:00:31.740 position:59.52%,start align:start size:13.81% line:10.00% \n<c.white><c.mono_sans>♪♪</c.mono_sans></c.white>']

## 불필요한 부분 제거

In [180]:
m = re.compile(r"\<.*?\>")
def clean(content):
    if '♪' in content: return None
    if '[' in content: return None
    if ']' in content: return None
    if '(' in content: return None
    if ')' in content: return None
    if '{' in content: return None
    if '}' in content: return None
    new_content = m.sub('',content)
    new_content = new_content.replace('-->','')
    remove_point_1 = new_content.find('position:')
    remove_point_2 = new_content.find('% \n')
    new_content = new_content[:remove_point_1] + new_content[remove_point_2+3:]
    return new_content

In [181]:
new_segments = [clean(s) for s in segments if len(s)!=0][4:]
new_segments[:10]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '12\n00:01:47.482  00:01:49.568 Boy: Mister?',
 '13\n00:01:49.610  00:01:51.236 Mister?']

## 줄번호 / 속도 시작 / 속도 끝 / 자막 - 데이터 분리

In [182]:
trimmed_segments = []
for segment in new_segments:
    if segment != None and segment.strip() != '':
      split_segment = segment.split()
      times = split_segment[:3]
      script = ' ' .join(split_segment[3:])
      times.append(script)
      trimmed_segments.append(times)

In [183]:
trimmed_segments[:5]

[['12', '00:01:47.482', '00:01:49.568', 'Boy: Mister?'],
 ['13', '00:01:49.610', '00:01:51.236', 'Mister?'],
 ['18', '00:02:26.271', '00:02:27.689', 'Woman: Ladies and gentlemen,'],
 ['19', '00:02:27.731', '00:02:29.733', 'welcome to San Jose'],
 ['20', '00:02:27.731', '00:02:29.733', 'International Airport.']]

## 데이터 저장

In [184]:
file_name = file_list[0].split('.S01')[0] + '.WEBRip.Netflix.en[cc]'
file_name

'The.Good.Doctor.WEBRip.Netflix.en[cc]'

In [185]:
with open(f'{path}/{file_name}.csv', 'w', encoding='utf8', newline='') as f:
    for line in trimmed_segments:
        thewriter = csv.writer(f)
        thewriter.writerow(line)

## 최종 - 줄번호 / 속도 시작 / 속도 끝 / 대사 별로 컬럼 분리

In [186]:
pd.set_option('display.max_colwidth', None)
data = pd.read_csv(f'{path}/{file_name}.csv', names=['num', 'start', 'end', 'script'])

data.head()

,num,start,end,script
0,12,00:01:47.482,00:01:49.568,Boy: Mister?
1,13,00:01:49.610,00:01:51.236,Mister?
2,18,00:02:26.271,00:02:27.689,"Woman: Ladies and gentlemen,"
3,19,00:02:27.731,00:02:29.733,welcome to San Jose
4,20,00:02:27.731,00:02:29.733,International Airport.


In [ ]:
if data['script'].iloc[-1][-1] not in ['.', '!', '?']:
    data['script'].iloc[-1] = data['script'].iloc[-1] + '.'

data.tail()

In [187]:
data['script'] = data['script'].str.replace('\"', '')

# 쪼개진 문장 합치기
for index, current_row in data.iterrows():
    if current_row['script'][-1] in ['.', '!', '?']:
        continue
    else:
        row_index = index + 1
    while True:
        next_row = data.loc[row_index]
        new_script = data.loc[index, 'script'] + ' ' + next_row['script']
        data.loc[index, 'script'] = new_script
        if next_row['script'][-1] in ['.', '!', '?']:
            data.loc[index, 'end'] = next_row['end']
            break
        else:
            row_index += 1

# 필요없는 토막문장들 제거
data = data.drop_duplicates(subset='end')
data.head(30)

,num,start,end,script
0,12,00:01:47.482,00:01:49.568,Boy: Mister?
1,13,00:01:49.610,00:01:51.236,Mister?
2,18,00:02:26.271,00:02:29.733,"Woman: Ladies and gentlemen, welcome to San Jose International Airport."
5,29,00:02:56.802,00:03:01.515,Look out! Watch out!
6,30,00:03:01.556,00:03:02.766,Help!
7,31,00:03:02.808,00:03:04.518,-Adam?!
9,33,00:03:04.559,00:03:06.020,Adam?!
10,34,00:03:06.061,00:03:09.230,Somebody call 911!
11,35,00:03:09.272,00:03:11.483,"Oh, my God!"
12,36,00:03:11.525,00:03:15.821,"I'm a doctor, let me take a look. Let me take a look."


In [188]:
data.to_csv(f'{path}/{file_name}.csv', columns=['start', 'end', 'script'], index=False)
data = pd.read_csv(f'{path}/{file_name}.csv')

data

,start,end,script
0,00:01:47.482,00:01:49.568,Boy: Mister?
1,00:01:49.610,00:01:51.236,Mister?
2,00:02:26.271,00:02:29.733,"Woman: Ladies and gentlemen, welcome to San Jose International Airport."
3,00:02:56.802,00:03:01.515,Look out! Watch out!
4,00:03:01.556,00:03:02.766,Help!
...,...,...,...
5779,00:42:02.770,00:42:05.190,"I mean, he's not at the hospital."
5780,00:42:05.231,00:42:10.111,And there used to be a picture here of Shaun...
5781,00:42:10.153,00:42:12.613,"and his brother, Steve."
5782,00:42:14.282,00:42:15.908,What do you think that means?
